# Classifying Fashion-MNIST

Now it's your turn to build and train a neural network. You'll be using the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist), a drop-in replacement for the MNIST dataset. MNIST is actually quite trivial with neural networks where you can easily achieve better than 97% accuracy. Fashion-MNIST is a set of 28x28 greyscale images of clothes. It's more complex than MNIST, so it's a better representation of the actual performance of your network, and a better representation of datasets you'll use in the real world.

<img src='assets/fashion-mnist-sprite.png' width=500px>

In this notebook, you'll build your own neural network. For the most part, you could just copy and paste the code from Part 3, but you wouldn't be learning. It's important for you to write the code yourself and get it to work. Feel free to consult the previous notebooks though as you work through this.

First off, let's load the dataset through torchvision.

In [1]:
# import torch
# from torchvision import datasets, transforms
# import helper

# # Define a transform to normalize the data
# transform = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize((0.5,), (0.5,))])
# # Download and load the training data
# trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# # Download and load the test data
# testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import seaborn as sns

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train = train['label']
x_train = train.drop(['label'], axis = 1)
y_test = test['label']
x_test = test.drop(['label'], axis = 1)

y_train.value_counts() 

In [ ]:
train_x = torch.tensor(x_train.values.astype(np.float32)) 
train_y = torch.tensor(y_train.values.astype(np.int64))
train_x[0], train_y[0]

In [ ]:
test_x = torch.tensor(x_test.values.astype(np.float32))
test_y = torch.tensor(y_test.values.astype(np.int64))

In [ ]:
class mydataset(Dataset):
    def __init__(self, data, labels):
        self.labels = labels
        self.data = data
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,i):
        item = self.data[i]
        label = self.labels[i]
        return (item,label)
    
training_data = mydataset(train_x,train_y)
testing_data = mydataset(test_x,test_y)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size = 100, shuffle = True)
test_dataloader = DataLoader(testing_data, batch_size = 100, shuffle = True)

Here we can see one of the images.

## Building the network

Here you should define your network. As with MNIST, each image is 28x28 which is a total of 784 pixels, and there are 10 classes. You should include at least one hidden layer. We suggest you use ReLU activations for the layers and to return the logits or log-softmax from the forward pass. It's up to you how many layers you add and the size of those layers.

In [ ]:
# TODO: Define your network architecture here
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.sequence = nn.Sequential(
            nn.Linear(784,100),
            nn.BatchNorm1d(num_features = 100), # normalize inputs across all the batches
            nn.ReLU(),
            nn.Dropout(0.4), #DropOut minimise the overfitting on training set
            nn.Linear(100,64),
            nn.BatchNorm1d(num_features = 64),
            nn.ReLU(),
            nn.Linear(64,28),
            nn.BatchNorm1d(num_features = 28),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(28,10),
            nn.LogSoftmax(dim=1)
        )
    def forward(self,x):
        output = self.sequence(x)
        return output


# Train the network

Now you should create your network and train it. First you'll want to define [the criterion](http://pytorch.org/docs/master/nn.html#loss-functions) ( something like `nn.CrossEntropyLoss`) and [the optimizer](http://pytorch.org/docs/master/optim.html) (typically `optim.SGD` or `optim.Adam`).

Then write the training code. Remember the training pass is a fairly straightforward process:

* Make a forward pass through the network to get the logits 
* Use the logits to calculate the loss
* Perform a backward pass through the network with `loss.backward()` to calculate the gradients
* Take a step with the optimizer to update the weights

By adjusting the hyperparameters (hidden units, learning rate, etc), you should be able to get the training loss below 0.4.

In [ ]:
# TODO: Create the network, define the criterion and optimizer
model = NN()
criterion = nn.CrossEntropyLoss()
epochs = 100
lr_rate = .005

optimizer = torch.optim.Adam(model.parameters(),lr = lr_rate)

In [ ]:
# TODO: Train the network here
def train_loop(dataloader, model, loss_fn, optimiser):
    size = len(dataloader.dataset)
    train_loss, correct = 0.0, 0
    for X, y in dataloader:
        pred = model(X)
        loss = criterion(pred, y)
        train_loss += loss.item() # Adding loss across all batches
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(pred, 1)
        correct += (predicted == y).sum().item()
        #Adding Accuracy across all batches
        
    train_loss /= size #mean of loss
    correct /= size  #mean of accuracy     
    print(f"Train Loss: {train_loss:.3f}")
    print(f"Train Accuracy: {correct:.3f}")
    return train_loss
    
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += criterion(pred, y).item()
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Loss: {test_loss:.3f}")
    print(f"Test Accuracy: {correct:.3f}")
    return test_loss

In [ ]:
total_train_loss = []
total_test_loss = []

for t in range(epochs):
    print(f"[=================== Epoch: {t} ==================]")
    train_los = train_loop(train_dataloader, model, criterion, optimizer)
    total_train_loss.append(train_los)
    print('')
    test_los = test_loop(test_dataloader, model, criterion)
    total_test_loss.append(test_los)

In [ ]:
plt.plot(total_train_loss)
plt.plot(total_test_loss)
plt.xlabel("No. of Iteration")
plt.ylabel("LOSS")
plt.show

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import helper

# Test out your network!

dataiter = iter(test_dataloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.resize(1, 784)

# TODO: Calculate the class probabilities (softmax) for img
ps = torch.exp(model(img))

# Plot the image and probabilities
helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')